In [1]:
import pyspark
from delta import configure_spark_with_delta_pip

# Prepare the Spark builder
builder = pyspark.sql.SparkSession.builder.appName('Project 3') \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder,extra_packages=["graphframes:graphframes:0.8.4-spark3.5-s_2.12"]).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)

spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

In [2]:
from graphframes import GraphFrame
from pyspark.sql.functions import col, coalesce, lit
from pyspark.sql.functions import sum as spark_sum

In [3]:
df = spark.read.csv('./data/2009.csv', header=True)

In [4]:
# airports
vertices = df.selectExpr("ORIGIN as id").union(df.selectExpr("DEST as id")).distinct()

# flight is a directed edge from start airport (ORIGIN) to destination airport (DEST)
edges = df.selectExpr("ORIGIN as src", "DEST as dst")

gf = GraphFrame(vertices, edges)

In [5]:
gf.persist()

GraphFrame(v:[id: string], e:[src: string, dst: string])

## Query 1

In [6]:
in_degree = gf.inDegrees

#in_degree.orderBy("inDegree", ascending=True).show()
in_degree.show()

+---+--------+
| id|inDegree|
+---+--------+
|JAX|   28813|
|BOS|  110463|
|MEM|   71721|
|XNA|   13764|
|BTV|    6021|
|SYR|    9330|
|JFK|  119571|
|MBS|    3443|
|SBN|    4527|
|HPN|   10661|
|EKO|    1416|
|ANC|   17788|
|VLD|    1046|
|DAB|    2149|
|EVV|    4761|
|BGM|     677|
|KTN|    2373|
|WRG|     722|
|MAF|    6526|
|MIA|   63949|
+---+--------+
only showing top 20 rows



In [7]:
out_degree = gf.outDegrees

#out_degree.orderBy("outDegree", ascending=True).show()
out_degree.show()

+---+---------+
| id|outDegree|
+---+---------+
|JAX|    28810|
|MEM|    71713|
|BTV|     6028|
|BOS|   110460|
|XNA|    13755|
|SYR|     9336|
|JFK|   119574|
|MBS|     3444|
|SBN|     4526|
|HPN|    10657|
|EKO|     1416|
|ANC|    17791|
|VLD|     1046|
|DAB|     2146|
|EVV|     4761|
|BGM|      678|
|KTN|     2374|
|WRG|      722|
|MAF|     6523|
|MIA|    63943|
+---+---------+
only showing top 20 rows



In [8]:
degree = in_degree.join(out_degree, "id", "outer") \
    .withColumn("inDegree", coalesce(in_degree["inDegree"], pyspark.sql.functions.lit(0))) \
    .withColumn("outDegree", coalesce(out_degree["outDegree"], pyspark.sql.functions.lit(0))) \
    .withColumn("totalDegree", pyspark.sql.functions.col("inDegree") + pyspark.sql.functions.col("outDegree"))

#degree.orderBy("totalDegree", ascending=True).show()
degree.show()

+---+--------+---------+-----------+
| id|inDegree|outDegree|totalDegree|
+---+--------+---------+-----------+
|ABE|    4037|     4034|       8071|
|ABI|    2490|     2490|       4980|
|ABQ|   35577|    35582|      71159|
|ABY|     997|      995|       1992|
|ACK|     343|      342|        685|
|ACT|    1052|     1053|       2105|
|ACV|    3364|     3370|       6734|
|ACY|     522|      522|       1044|
|ADK|     103|      103|        206|
|ADQ|     631|      631|       1262|
|AEX|    2948|     2947|       5895|
|AGS|    3106|     3107|       6213|
|AKN|      77|       77|        154|
|ALB|   12020|    12018|      24038|
|ALO|     331|      330|        661|
|AMA|    6649|     6649|      13298|
|ANC|   17788|    17791|      35579|
|ASE|    4708|     4701|       9409|
|ATL|  417457|   417449|     834906|
|ATW|    5306|     5303|      10609|
+---+--------+---------+-----------+
only showing top 20 rows



In [9]:
triangles = gf.triangleCount() # triangles that include each vertex
# triangles.select("id", "count").orderBy("count", ascending=True).show()
triangles.select("id", "count").show()

+---+-----+
| id|count|
+---+-----+
|JAX|  342|
|MEM| 1105|
|BTV|   34|
|BOS|  860|
|XNA|   97|
|SYR|   82|
|JFK|  942|
|MBS|    6|
|SBN|   13|
|HPN|   36|
|EKO|    0|
|ANC|  122|
|VLD|    0|
|DAB|    1|
|EVV|   15|
|BGM|    0|
|KTN|    4|
|WRG|    2|
|BJI|    0|
|SAF|    1|
+---+-----+
only showing top 20 rows



## Query 2

In [10]:
total_triangles = triangles.select(spark_sum("count")).first()[0] # divide by 3 as each triangle has 3 airports

print(f"Total number of triangles: {total_triangles}")

Total number of triangles: 48045


## Query 3

In [11]:
# EigenCentrality
## https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/

# Make undirected edges by adding reverse direction
reversed_edges = edges.selectExpr("dst as src", "src as dst")
undirected_edges = edges.union(reversed_edges).distinct()

scores = vertices.withColumn("score", lit(1.0))

In [12]:
num_iterations = 10

for i in range(num_iterations):
    # Join edges with current scores
    joined = undirected_edges.join(scores, undirected_edges.dst == scores.id, how="left") \
                              .select(undirected_edges.src.alias("id"), col("score"))

    # Sum neighbor scores to update each node
    updated_scores = joined.groupBy("id").agg(spark_sum("score").alias("score"))

    # Normalize scores to prevent divergence (optional)
    total = updated_scores.agg(spark_sum("score")).first()[0]
    normalized_scores = updated_scores.withColumn("score", col("score") / total)

    # Update for next iteration
    scores = normalized_scores

In [13]:
scores.orderBy(col("score").desc()).show()

+---+--------------------+
| id|               score|
+---+--------------------+
|ATL|0.018765722188918116|
|ORD|0.018108878438614258|
|DFW|0.016941191132151725|
|DTW| 0.01669891075140396|
|DEN|0.016140351072366586|
|MSP|0.016124197587719383|
|IAH|0.015634522037957406|
|CVG|0.015054412087523953|
|EWR|0.014559829904750299|
|LAS|0.014527065196091225|
|PHX|0.014275998919980493|
|MEM|0.014028033193475667|
|MCO|0.013546284087077538|
|CLT|0.013532499671015705|
|LAX| 0.01323284686945521|
|SLC|0.013181054625054962|
|BWI|0.013131956958904989|
|IAD|0.013019495597477422|
|CLE|  0.0129345700134493|
|PHL|0.012915917719506932|
+---+--------------------+
only showing top 20 rows



## Query 4

In [14]:
pagerank_results = gf.pageRank( # calculate the importance of an airport
    resetProbability=0.15,
    maxIter=5,
)

pagerank_results.vertices.select("id", "pagerank").orderBy("pagerank", ascending=False).show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|ATL| 19.50526915337207|
|ORD|13.202402918484228|
|DFW|12.006642541639124|
|DEN|10.182853592388355|
|LAX| 7.675967810587112|
|IAH|7.2916990138245215|
|SLC| 7.210595384167181|
|DTW| 7.133956148107741|
|PHX| 7.088024515578732|
|SFO| 5.900934855935846|
|MSP|5.7980572416857905|
|LAS| 5.654682552839485|
|SEA| 5.244648319039893|
|JFK| 4.577452212441765|
|MCO| 4.388433951465143|
|EWR|  4.33149433800225|
|CLT| 4.202519749376691|
|BOS| 3.790060795120423|
|BWI|3.5786305372666014|
|LGA| 3.550321125760009|
+---+------------------+
only showing top 20 rows



## Query 5

In [15]:
# Degree centrality (number of direct connections)
## Shows how connected an airport is
## Can be used to detect big airports or bottlenecks

degree_centrality = in_degree.join(out_degree, "id", "outer") \
    .withColumn("inDegree", coalesce(col("inDegree"), lit(0))) \
    .withColumn("outDegree", coalesce(col("outDegree"), lit(0))) \
    .withColumn("degreeCentrality", col("inDegree") + col("outDegree"))


degree_centrality.orderBy(col("degreeCentrality").desc()).show()

+---+--------+---------+----------------+
| id|inDegree|outDegree|degreeCentrality|
+---+--------+---------+----------------+
|ATL|  417457|   417449|          834906|
|ORD|  313769|   313848|          627617|
|DFW|  264398|   264396|          528794|
|DEN|  235700|   235675|          471375|
|LAX|  192916|   192879|          385795|
|PHX|  183491|   183502|          366993|
|IAH|  182088|   182097|          364185|
|LAS|  153984|   153993|          307977|
|DTW|  152075|   152081|          304156|
|SFO|  136532|   136488|          273020|
|SLC|  131674|   131694|          263368|
|MCO|  120936|   120944|          241880|
|MSP|  119759|   119732|          239491|
|JFK|  119571|   119574|          239145|
|EWR|  118602|   118602|          237204|
|CLT|  116640|   116650|          233290|
|BOS|  110463|   110460|          220923|
|SEA|  100922|   100948|          201870|
|BWI|  100928|   100923|          201851|
|LGA|  100323|   100334|          200657|
+---+--------+---------+----------